In [1]:
#Memasang library yang dibutuhkan
import sys
import psycopg2
import pandas as pd

In [2]:
#koneksi database
conn = psycopg2.connect(host="localhost", port = 5432, database="dsda_masgading", user="postgres", password="sectio14")
conn.set_session(autocommit=True)
cur = conn.cursor()
sql = "SELECT * FROM public.orders"
cur.execute(sql)
baris = cur.fetchall()

In [3]:
#Listing raw dataset, rename, dan menampilkan n data
order = pd.DataFrame([[ij for ij in i]for i in baris])
order.rename(columns={0:'order_id', 1:'customer_id',2:'campaign_id', 3:'order_date', 4:'city', 5:'state', 6:'zipcode', 7:'payment_type', 8:'total_price', 9:'num_order', 10:'num_units'}, inplace = True)
order.head(4)

,order_id,customer_id,campaign_id,order_date,city,state,zipcode,payment_type,total_price,num_order,num_units
0,1002854,45978,2141,2009-10-13,NEWTON,MA,2459,VI,190.00,3,3
1,1002855,125381,2173,2009-10-13,NEW ROCHELLE,NY,10804,VI,10.00,1,1
2,1002856,103122,2141,2011-06-02,MIAMI,FL,33137,AE,35.22,2,2
3,1002857,130980,2173,2009-10-14,E RUTHERFORD,NJ,7073,AE,10.00,1,1


In [4]:
#Cek missing value/null
print(order.isnull().sum())

order_id           0
customer_id        0
campaign_id        0
order_date         0
city              17
state           1119
zipcode          144
payment_type       0
total_price        0
num_order          0
num_units          0
dtype: int64


In [5]:
#Fix missing value/null dan cek missing value/null setelah dibersihkan
cleandata = order.dropna()

In [6]:
#Fix missing value/null dan cek missing value/null setelah dibersihkan
print(cleandata.isnull().sum())

order_id        0
customer_id     0
campaign_id     0
order_date      0
city            0
state           0
zipcode         0
payment_type    0
total_price     0
num_order       0
num_units       0
dtype: int64


In [7]:
#Cek dimensi raw data
order.shape

(192983, 11)

In [8]:
#Cek dimensi data yang sudah bersih
cleandata.shape

(191848, 11)

In [9]:
#Tampilkan n data yang sudah bersih
cleandata.head(4)

,order_id,customer_id,campaign_id,order_date,city,state,zipcode,payment_type,total_price,num_order,num_units
0,1002854,45978,2141,2009-10-13,NEWTON,MA,2459,VI,190.00,3,3
1,1002855,125381,2173,2009-10-13,NEW ROCHELLE,NY,10804,VI,10.00,1,1
2,1002856,103122,2141,2011-06-02,MIAMI,FL,33137,AE,35.22,2,2
3,1002857,130980,2173,2009-10-14,E RUTHERFORD,NJ,7073,AE,10.00,1,1


In [10]:
#Manipulasi berdasarkan grup data
data = order[['order_id','customer_id','campaign_id','total_price']]
idOrder = data.groupby('campaign_id')['customer_id'].nunique()
print('Jumlah Customer:\n',idOrder)

Jumlah Customer:
 campaign_id
2001        5
2002       91
2003      276
2004        3
2005       22
        ...  
2235      990
2236    50760
2237     1129
2238        1
2239        4
Name: customer_id, Length: 239, dtype: int64


In [11]:
#Manipulasi berdasarkan filter data NY
man2 = cleandata[['order_id','customer_id','campaign_id','payment_type']]
stateNY = man2['campaign_id'].loc[cleandata['state']=="NY"]
print(stateNY.value_counts())

2141    22691
2173    11274
2236    10634
2010     1460
2237     1183
        ...  
2008        1
2220        1
2132        1
2123        1
2147        1
Name: campaign_id, Length: 182, dtype: int64


In [12]:
#Manipulasi berdasarkan filter data PA
statePA = man2['campaign_id'].loc[cleandata['state']=="PA"]
print(statePA.value_counts())

2141    2524
2236    1985
2173    1645
2010     159
2204     127
        ... 
2184       1
2033       1
2185       1
2012       1
2028       1
Name: campaign_id, Length: 99, dtype: int64


In [13]:
#Normalisasi Zscore (Sebelum)
zdata = cleandata.loc[:,['total_price']]
zdata['total_price'] = zdata['total_price'].fillna(cleandata.groupby('campaign_id')['total_price'].transform('mean'))
print('Data Sebelum normalisasi ZScore:\n',zdata)

Data Sebelum normalisasi ZScore:
         total_price
0            190.00
1             10.00
2             35.22
3             10.00
4             10.00
...             ...
192978        23.96
192979        20.65
192980        16.95
192981        22.95
192982        49.45

[191848 rows x 1 columns]


In [14]:
#Normalisasi Zscore (Sesudah)
avgdata = zdata.mean()
stddata =  zdata.std()
zdata = (zdata-avgdata)/stddata
print('Data setelah normalisasi ZScore:\n',zdata)

Data setelah normalisasi ZScore:
         total_price
0          0.652530
1         -0.333498
2         -0.195345
3         -0.333498
4         -0.333498
...             ...
192978    -0.257026
192979    -0.275158
192980    -0.295426
192981    -0.262559
192982    -0.117394

[191848 rows x 1 columns]


In [15]:
#Normalisasi Minmax (Sebelum)
mdata = cleandata.loc[:,['total_price']]
mdata['total_price'] = mdata['total_price'].fillna(cleandata.groupby('campaign_id')['total_price'].transform('mean'))
print('Data sebelum normalisasi minmax\n',data)

Data sebelum normalisasi minmax
        order_id customer_id campaign_id  total_price
0       1002854       45978        2141       190.00
1       1002855      125381        2173        10.00
2       1002856      103122        2141        35.22
3       1002857      130980        2173        10.00
4       1002886       48553        2141        10.00
...         ...         ...         ...          ...
192978  1643147      188940        2170        23.96
192979  1643148      189550        2141        20.65
192980  1643149      188938        2236        16.95
192981  1643150      189517        2236        22.95
192982  1643157      167434        2141        49.45

[192983 rows x 4 columns]


In [16]:
#Normalisasi Minmax (Sesudah)
min = 0
max = 1
mindata = mdata.min()
maxdata = mdata.max()
mdata = ((mdata-mindata)*(max-min)/(maxdata-mindata))+min
print('Data setelah normalisasi minmax:\n',mdata)

Data setelah normalisasi minmax:
         total_price
0          0.019291
1          0.001015
2          0.003576
3          0.001015
4          0.001015
...             ...
192978     0.002433
192979     0.002097
192980     0.001721
192981     0.002330
192982     0.005021

[191848 rows x 1 columns]


In [17]:
data1 = order[['order_id','customer_id','payment_type','total_price']]
customerid = data1.groupby('payment_type')['customer_id'].nunique()
print('Jumlah Customer:\n',customerid)

Jumlah Customer:
 payment_type
??      304
AE    46791
DB    12438
MC    46584
OC     8158
VI    75290
Name: customer_id, dtype: int64


In [18]:
data2 = order[['order_id','customer_id','city','total_price']]
customerid = data2.groupby('city')['customer_id'].nunique()
print('Jumlah Customer:\n',customerid)

Jumlah Customer:
 city
#07-01 FONTANA HGTS     1
#36 NYE                 2
.EASTHAM                2
0201 OSLO               1
05446-030               1
                       ..
ZIONSDILLE              2
ZIONSVILLE             22
ZIONSVIOLE              1
ZOLLIKERBERG            2
ZURICH                  1
Name: customer_id, Length: 12824, dtype: int64


In [24]:
#Manipulasi berdasarkan filter data TX
man3 = cleandata[['order_id','customer_id','num_units','payment_type']]
stateTX = man3['num_units'].loc[cleandata['state']=="TX"]
print(stateTX.value_counts())

1      4117
2      1252
3       532
4       175
5        75
6        30
7        24
10       16
8         7
11        6
9         3
15        3
13        2
20        2
25        1
16        1
101       1
Name: num_units, dtype: int64


In [25]:
#Manipulasi berdasarkan filter data FL
man4 = cleandata[['order_id','customer_id','num_order','payment_type']]
stateFL = man4['num_order'].loc[cleandata['state']=="FL"]
print(stateFL.value_counts())

1     7366
2     1684
3      695
4      198
5       99
6       59
7       31
8       18
9       15
11       7
10       4
12       3
22       2
20       1
14       1
13       1
Name: num_order, dtype: int64
